In [13]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc-julia/state_constraints/Project.toml`


In [14]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [15]:
# mass_scale = 1/.5
# length_scale = 1/.1750
# time_scale = 1/.05

mass_scale = 1
length_scale = 1
time_scale = 1

1

In [16]:
# # Quadrotor parameters
# m = 0.035
# ℓ = 0.046/sqrt(2)
# J = [16.6e-6 0.83e-6 0.72e-6;
#     0.83e-6 16.6e-6 1.8e-6;
#     0.72e-6 1.8e-6 29.3e-6];
# gravity = 9.81
# # thrustToTorque = 0.005964552
# thrustToTorque = 0.0008
# scale = 65535
# kt = 2.245365e-6*scale # u is PWM in range [0...1]
# # kt=1
# km = kt*thrustToTorque #4.4733e-8

# h = 1/500

# Nx = 13     # number of states (quaternion)
# Nx̃ = 12     # number of states (linearized)
# Nu = 4      # number of controls
# nx = Nx̃
# nu = Nu


#Quadrotor parameters
m = 0.5 * mass_scale # kg
ℓ = 0.1750 * length_scale # meters
J = Diagonal([0.0023, 0.0023, 0.004]) * mass_scale * length_scale^2 # kg * meters^2
gravity = 9.81 * length_scale / (time_scale^2)
kt = 1.0 * mass_scale * length_scale / (time_scale^2) # N
km = 0.0245 * mass_scale * length_scale^2 / (time_scale^2) # Nm

h = 0.05 # 20 Hz

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4     # number of controls
nx = Nx̃
nu = Nu

#Goal state
@show uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*gravity/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * gravity) / kt) / 4) * ones(4) = [1.22625, 1.22625, 1.22625, 1.22625]


In [17]:
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    v = x[8:10]
    ω = x[11:13]
    Q = qtoQ(q)
    
    ṙ = v
    q̇ = 0.5*L(q)*H*ω
    
    b = 1/m
  
    v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 
  
    ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
    
    return [ṙ; q̇; v̇; ω̇]
  end
  function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
  end

quad_dynamics_rk4 (generic function with 1 method)

In [18]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [19]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

# Create obstacle in MeshCat
# Cylinder constraint
c = [0.0; 2.0]
r_vis = 0.1
r_obstacle = 0.4
setobject!(vis["cylinder"], Cylinder(Point(c[1], c[2], 0.0), Point(c[1], c[2], 1.0), r_vis))
bounding_cylinder_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
setobject!(vis["bounding cylinder"], Cylinder(Point(c[1], c[2], -.05), Point(c[1], c[2], 1.05), r_obstacle), bounding_cylinder_material)

function TransformPlane(visObject, center, dir)
    # dir[1] = x, dir[2] = y (for three dimensions, take cross product of [1,0,0] and 3D dir vector to get axis of rotation)
    width = 2
    height = 1
    dir_inv = -[dir[2]; -dir[1]]/norm(dir)
    bl = [center[1:2] + dir_inv*width/2; 0] # bottom left corner
    settransform!(visObject, Translation(bl[1], bl[2], bl[3]) ∘ LinearMap(AngleAxis(-atan(dir[2], -dir[1]), 0, 0, 1)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [24]:
Tfinal = 5.0            # final time
N = Int(Tfinal/h)+1     # number of time steps
t_vec = h*(0:N-1)

# Cost weights
# Q = Array(BlockDiagonal([5.0*I(3), 0.001*I(Nx̃-3)]));
# R = Array(1*I(Nu));
# Qf = 1*Q;


# Cost weights
max_dev_x = [0.01; 0.01; 0.01;  0.5; 0.5; 0.5;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
# max_dev_u = [0.5; 0.5; 0.5; 0.5]
max_dev_u = [1; 1; 1; 1]
Q = diagm(1 ./(max_dev_x.^2))
R = diagm(1 ./(max_dev_u.^2))
Qf = 1*Q

# # Cost weights
# Q = Array(I(Nx̃));
# R = Array(.1*I(Nu));
# Qf = 1*Q

# Penalty
ρ = 5.0
R̃ = R + ρ*I;
# R̃ = R

# Precompute
cache = (
    A = deepcopy(Ãdyn),
    B = deepcopy(B̃dyn),
    Kinf = zeros(nu,nx),
    Pinf = zeros(nx,nx),
    Quu_inv = zeros(nu,nu),
    AmBKt = zeros(nx,nx), 
    coeff_d2p = zeros(nx,nu), 
    Qu1 = zeros(nu,nx), 
    Qu2 = zeros(nu,nu),
    Kt = zeros(nx,nu),
)

Nriccati = 1000
P = [zeros(nx,nx) for i = 1:Nriccati];   # cost to go quadratic term
K = [zeros(nu,nx) for i = 1:Nriccati-1]; # feedback gain

P[Nriccati] .= Qf;
for k = (Nriccati-1):-1:1
    K[k] = (R̃ + B̃dyn'*P[k+1]*B̃dyn)\(B̃dyn'*P[k+1]*Ãdyn);
    P[k] = Q + Ãdyn'*P[k+1]*(Ãdyn - B̃dyn*K[k]);
end

Kinf_float = deepcopy(K[1]);
Pinf_float = deepcopy(P[1]);
Quu_inv_float = (R̃ + cache.B'*cache.Pinf*cache.B)\I;
AmBKt_float = (cache.A - cache.B*cache.Kinf)';
coeff_d2p_float = cache.Kinf'*R̃ - cache.AmBKt*cache.Pinf*cache.B;


# Create trajectory to follow
X̃ref = [zeros(nx) for i = 1:N]
Uref = [zeros(nu) for i = 1:N-1]

scale = 1
loops = 1

# Zero
# Xref = [[zeros(3); 1; zeros(9)] for _ = 1:N];

# Figure-eight
# Xref = [[scale*sin(2*t)/2; scale*cos(t); 1; 1; zeros(9)] for t = range(-pi/2*loops, 3*pi/2*loops, length = N)];

# Squished pringle
# Xref = [[scale*sin(2*t)/2; scale*cos(t); 0.3*sin(t)+1; 1; zeros(9)] for t = range(-pi/2*loops, 3*pi/2*loops, length = N)];

# Line
Xref = [[0.01; t; 0.5; 1; zeros(9)] for t = range(0, 4, length = N)];

# Compute reference velocity from reference position
for i = 1:N-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [25]:
#Fixed-Point Conversion

r_max = 10000.0
q_max = 10.0
ϕ_max = 30.0
v_max = 100.0
ω_max = 200.0
control_max = 0.5*m*gravity

fp_max = 2147483647.0
# fp_max = 214748.0

J_xfp = Diagonal([ones(3)*(r_max); ones(3)*(ϕ_max); ones(3)*(v_max); ones(3)*(ω_max)])
J_ufp = Diagonal(ones(4)*(control_max))

J_xfp_inv = J_xfp\I
J_ufp_inv = J_ufp\I

function state_float_to_fixed(x)
    x_fixed = zeros(Int32, 13)
    x_fixed[1:3] .= convert(Vector{Int32},round.(x[1:3].*(fp_max/r_max)))
    x_fixed[4:7] .= convert(Vector{Int32},round.(x[4:7].*(fp_max/q_max)))
    x_fixed[8:10] .= convert(Vector{Int32},round.(x[8:10].*(fp_max/v_max)))
    x_fixed[11:13] .= convert(Vector{Int32},round.(x[11:13].*(fp_max/ω_max)))
    
    return x_fixed
end

function Δstate_float_to_fixed(x)
    x_fixed = zeros(Int32, 12)
    x_fixed[1:3] .= convert(Vector{Int32},round.(x[1:3].*(fp_max/r_max)))
    x_fixed[4:6] .= convert(Vector{Int32},round.(x[4:6].*(fp_max/ϕ_max)))
    x_fixed[7:9] .= convert(Vector{Int32},round.(x[7:9].*(fp_max/v_max)))
    x_fixed[10:12] .= convert(Vector{Int32},round.(x[10:12].*(fp_max/ω_max)))
    
    return x_fixed
end

function state_fixed_to_float(x)
    x_float = zeros(13)
    x_float[1:3] .= convert(Vector{Float64},x[1:3]).*(r_max/fp_max)
    x_float[4:7] .= convert(Vector{Float64},x[4:7]).*(q_max/fp_max)
    x_float[8:10] .= convert(Vector{Float64},x[8:10]).*(v_max/fp_max)
    x_float[11:13] .= convert(Vector{Float64},x[11:13]).*(ω_max/fp_max)
    
    return x_float
end

function Δstate_fixed_to_float(x)
    x_float = zeros(12)
    x_float[1:3] .= convert(Vector{Float64},x[1:3]).*(r_max/fp_max)
    x_float[4:6] .= convert(Vector{Float64},x[4:6]).*(ϕ_max/fp_max)
    x_float[7:9] .= convert(Vector{Float64},x[7:9]).*(v_max/fp_max)
    x_float[10:12] .= convert(Vector{Float64},x[10:12]).*(ω_max/fp_max)
    
    return x_float
end

function controls_float_to_fixed(u)
    u_fixed = zeros(Int32, 4)
    u_fixed[1:4] .= convert(Vector{Int32},round.((u[1:4]).*(fp_max/control_max)))
    # u_fixed[1:4] .= convert(Vector{Int32},(u[1:4]).*(fp_max/control_max))
    
    return u_fixed
end

function controls_fixed_to_float(u)
    u_float = zeros(4)
    u_float[1:4] .= convert(Vector{Float64}, u).*(control_max/fp_max)
    
    return u_float
end

controls_fixed_to_float (generic function with 1 method)

In [1]:
include("../tinyMPC-fixedpt-state.jl")

# Compute fixed point versions of cached matrices used in ADMM
Ã_fixed = round.(J_xfp_inv*Ãdyn*J_xfp*1000)
B̃_fixed = round.(J_xfp_inv*B̃dyn*J_ufp*1000)
Kinf_fixed = round.(J_ufp_inv*K[1]*J_xfp)
Pinf_fixed = round.(J_xfp_inv*P[1]*J_xfp)

Qu1 = round.(J_ufp_inv*(R̃ + B̃dyn'*P[1]*B̃dyn)^(-1)*B̃dyn'*J_xfp*1000)
Qu2 = round.(J_ufp_inv*(R̃ + B̃dyn'*P[1]*B̃dyn)^(-1)*J_ufp*100)

AmBKt_fixed = round.(J_xfp_inv*(Ãdyn - B̃dyn*K[1])'*J_xfp*100)
coeff_d2p_fixed1 = K[1]'*R̃
coeff_d2p_fixed2 = - (Ãdyn - B̃dyn*K[1])'*P[1]*B̃dyn
coeff_d2p_fixed = round.(J_xfp_inv*(coeff_d2p_fixed1 + coeff_d2p_fixed2)*J_ufp)
Kt_fixed = round.(J_xfp_inv*K[1]'*J_ufp*1000)

# Save fixed point version of cached matrices
cache.A .= Ã_fixed
cache.B .= B̃_fixed
cache.Kinf .= Kinf_fixed
cache.Pinf .= Pinf_fixed
cache.Qu1 .= Qu1
cache.Qu2 .= Qu2
# cache.Quu_inv .= Quu_inv_fixed
cache.AmBKt .= AmBKt_fixed
cache.coeff_d2p .= coeff_d2p_fixed
cache.Kt .= Kt_fixed

# Compute fixed point versions of parameters used by ADMM
ρ_fixed = round.(J_ufp_inv*ρ*J_ufp)

Q_fixed = round.(J_xfp_inv*Q*J_xfp)
R_fixed = round.(J_ufp_inv*R*J_ufp)
Qf_fixed = round.(J_xfp_inv*Qf*J_xfp)

# Convert X̃ref and Uref to fixed point
X̃ref_fixed = [Δstate_float_to_fixed(x_) for x_ in X̃ref]
Uref_fixed = [controls_float_to_fixed(u_) for u_ in Uref]


speye(N) = spdiagm(ones(N))

# MPC setup
Nh = 20

skip_vis = 5

# Create constraint planes
thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:skip_vis:Nh]


# Instantiate TinyMPC-ADMM variables

x = [zeros(Int32, nx) for i = 1:Nh]
u = [zeros(Int32, nu) for i = 1:Nh-1]

# State slack (auxiliary) and dual variables
v = [zeros(Int32, nx) for i = 1:Nh]
vnew = [zeros(Int32, nx) for i = 1:Nh]
g = [zeros(Int32, nx) for i = 1:Nh]

# Input slack (auxiliary) and dual variables
z = [zeros(Int32, nu) for i = 1:Nh-1]
znew = [zeros(Int32, nu) for i = 1:Nh-1]
y = [zeros(Int32, nu) for i = 1:Nh-1]

q = [zeros(Int32, nx) for i = 1:Nh];
r = [zeros(Int32, nu) for i = 1:Nh-1];

p = [zeros(Int32, nx) for i = 1:Nh];       # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(Int32, nu) for i = 1:Nh-1];     # feedforward control

# Input constraints
umin_fixed = controls_float_to_fixed(-[1.0; 1.0; 1.0; 1.0]*.5)
umax_fixed = controls_float_to_fixed([1.0; 1.0; 1.0; 1.0]*.5)

# State constraints
xmin_fixed = [-Inf .* ones(nx) for i = 1:Nh]
xmax_fixed = [Inf .* ones(nx) for i = 1:Nh]

Aineq = [speye(nx) for i = 1:Nh]

for k = 1:skip_vis:Nh
    x_this = X̃ref[k][1:2]
    xc = c - x_this
    a = xc/norm(xc)

    Aineq[k][1:2,1:2] = [a'; 0 0]

    q_c = c - r_obstacle*a
    b = a'*q_c
    # xmax[k][1] = b
    TransformPlane(constraintPlanes[cld(k,skip_vis)], q_c, a)
end

# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = Xref[1]
# xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]


for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state
# for k = 1:1
    x[1] = Δstate_float_to_fixed([xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]])

    params = (
        N = Nh,
        Q = Q_fixed, R = R_fixed, Qf = Qf_fixed,
        Xref = X̃ref_fixed[k:k+Nh-1], Uref = Uref_fixed[k:k+Nh-2], 
        cache = cache,
        fixed_A_divisor=1000,
        fixed_B_divisor=1000,
        
        A = Aineq,
        umin = umin_fixed, umax = umax_fixed,
        xmin = xmin_fixed, xmax = xmax_fixed,
    );

    # Solve
    Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; ρ=ρ_fixed, abs_tol=J_ufp_inv[1,1]*fp_max/100, max_iter=100)
    # @show iter, status


    # Roll out x_k+1 with actual dynamics
    uhist[k] = controls_fixed_to_float(Δu[1]) + uhover
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
        
    # Roll out with real dynamics
    x_real = [zeros(nx+1) for _ = 1:Nh]
    x_real[1] = xhist[k]
    for j = 1:Nh-1
        x_real[j+1] = quad_dynamics_rk4(x_real[j], uhist[k])
    end

    # Update linearized cylinder constraints
    Aineq = [speye(nx) for i = 1:Nh]
    xmax = [Inf .* ones(nx) for i = 1:Nh]
    for j = 1:skip_vis:Nh
        x_this = x_real[j][1:2]
        xc = c - x_this
        a = xc/norm(xc)

        Aineq[j][1:2,1:2] = [a'; 0 0]

        q_c = c - r_obstacle*a
        b = a'*q_c
        # xmax[j][1] = b
        TransformPlane(constraintPlanes[cld(j,skip_vis)], q_c, a)
    end

    # Visualize solution x
    x_float = [Δstate_fixed_to_float(x_) for x_ in x];
    delete!(vis["xLinearized"])
    x_linearized = [Point(x_[1], x_[2], x_[3]+0.01) for x_ in x_float]
    setobject!(vis["xLinearized"], Object(PointCloud(x_linearized), 
    LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))
end

delete!(vis["xHistline"])
xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

LoadError: LoadError: LoadError: UndefVarError: @sprintf not defined
in expression starting at /home/sam/Git/tinympc-julia/tinyMPC-fixedpt-state.jl:110
in expression starting at /home/sam/Git/tinympc-julia/tinyMPC-fixedpt-state.jl:104

In [28]:
# X1 = [SVector{13}([x_[1:3]; rptoq(x_[4:6]); x_[7:9]; x_[10:12]]) for x_ in x];
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)

In [75]:
display("cache.Qu1")
display(cache.Qu1)
display("cache.Qu2")
display(cache.Qu2)
display("cache.AmBKt")
display(cache.AmBKt)
display("cache.coeff_d2p")
display(cache.coeff_d2p)
display("cache.Kt")
display(cache.Kt)

display("cache.Kinf")
display(cache.Kinf)
display("cache.Pinf")
display(cache.Pinf)
display("cache.A")
display(cache.A)
display("cache.B")
display(cache.B)

display("rho")
display(ρ)

display("params.Q")
display(params.Q)
display("params.R")
display(params.R)

"cache.Qu1"

4×12 Matrix{Float64}:
 -0.0   0.0  7.0  -2.0  -2.0  -5.0  -2.0  …  273.0  -989.0  -989.0  -2476.0
  0.0   0.0  7.0  -2.0   2.0   5.0   2.0     273.0  -989.0   989.0   2476.0
  0.0  -0.0  7.0   2.0   2.0  -5.0   2.0     273.0   989.0   989.0  -2476.0
 -0.0  -0.0  7.0   2.0  -2.0   5.0  -2.0     273.0   989.0  -989.0   2476.0

"cache.Qu2"

4×4 Matrix{Float64}:
  4.0  -1.0   4.0  -1.0
 -1.0   4.0  -1.0   4.0
  4.0  -1.0   4.0  -1.0
 -1.0   4.0  -1.0   4.0

"cache.AmBKt"

...

12×4 Matrix{Float64}:
  -0.0    0.0   0.0   -0.0
   0.0    0.0  -0.0   -0.0
   0.0    0.0   0.0    0.0
  -4.0   -4.0   4.0    4.0
  -4.0    4.0   4.0   -4.0
  -0.0    0.0  -0.0    0.0
  -0.0    0.0   0.0   -0.0
   0.0    0.0  -0.0   -0.0
   2.0    2.0   2.0    2.0
 -47.0  -47.0  47.0   47.0
 -47.0   47.0  47.0  -47.0
  -4.0    4.0  -4.0    4.0

"rho"

5.0

"params.Q"

UndefVarError: UndefVarError: params not defined